## Load Libraries and Firmware

In [ ]:
# jupyter setup boilerplate
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from qick import *

# for now, all the tProc v2 classes need to be individually imported (can't use qick.*)

# the main program class
from qick.asm_v2 import AveragerProgramV2
# for defining sweeps
from qick.asm_v2 import QickSpan, QickSweep1D

In [ ]:
soc = QickSoc('/home/xilinx/jupyter_notebooks/fw/2025-06-15_216_tprocv2r24_standard/qick_216.bit')
soccfg = soc
print(soccfg)


## basic multi-pulse program
Just a "reps" loop, no sweeping - like v1 AveragerProgram.

In v2, you define all of your pulses, and then you can play them as needed. Contrast to v1, where only one pulse could be defined on a generator at a time.

In [ ]:
GEN_CH = 0
RO_CH = 0
TRIG_TIME = 0.40
FREQ = 100
# FREQ = 0

class MultiPulseProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        
        self.declare_gen(ch=gen_ch, nqz=1)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])

        ramp_len = 0.2
        self.add_gauss(ch=gen_ch, name="ramp", sigma=ramp_len/10, length=ramp_len, even_length=True)
        
        self.add_pulse(ch=gen_ch, name="myflattop", ro_ch=ro_ch, 
                       style="flat_top", 
                       envelope="ramp", 
                       freq=cfg['freq'], 
                       length=0.1,
                       phase=0,
                       gain=1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="mygaus", ro_ch=ro_ch, 
                       style="arb", 
                       envelope="ramp", 
                       freq=cfg['freq'], 
                       phase=0,
                       gain=1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="myconst", ro_ch=ro_ch, 
                       style="const", 
                       length=0.2, 
                       freq=cfg['freq'], 
                       phase=0,
                       gain=1.0,
                      )

        self.add_pulse(ch=gen_ch, name="myflattop2", ro_ch=ro_ch, 
                       style="flat_top", 
                       envelope="ramp", 
                       freq=cfg['freq'], 
                       length=0.1,
                       phase=90,
                       gain=1.0, 
                      )

        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['freq'], gen_ch=gen_ch)
        # send the config to the dynamic RO
        self.send_readoutconfig(ch=ro_ch, name="myro", t=0)
        
    def _body(self, cfg):
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'], ddr4=True)

        self.pulse(ch=cfg['gen_ch'], name="myflattop", t=0)
        self.pulse(ch=cfg['gen_ch'], name="mygaus", t=0.4)
        self.pulse(ch=cfg['gen_ch'], name="myconst", t=0.8)
        self.pulse(ch=cfg['gen_ch'], name="myflattop2", t=1.2)
        self.pulse(ch=cfg['gen_ch'], name="mygaus", t=1.6)
        
config = {'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'freq': FREQ,
          'trig_time': TRIG_TIME,
          'ro_len': 1.9,
         }

prog = MultiPulseProgram(soccfg, reps=1, final_delay=0.5, cfg=config)

iq_list = prog.acquire_decimated(soc, rounds=10)
t = prog.get_time_axis(ro_index=0)

plt.plot(t, iq_list[0][:,0], label="I value")
plt.plot(t, iq_list[0][:,1], label="Q value")
plt.plot(t, np.abs(iq_list[0].dot([1,1j])), label="magnitude")
plt.legend()
plt.ylabel("a.u.")
plt.xlabel("us")

print(prog)
prog.print_pmem2hex()
prog.print_wmem2hex()
prog.print_sg_mem(sg_idx=0,gen_file=True)

## tProc basic functionality
Write Data Registers

In [ ]:
from qick.asm_v2 import QickProgramV2

prog = QickProgramV2(soccfg)
prog.declare_readout(ch=0, length=100)
prog.label('reg_wr_test')
# Write all the Data registers with some value
prog.write_reg(dst='r0',  src=12345678)
prog.write_reg(dst='r1',  src=12345678)
prog.write_reg(dst='r2',  src=12345678)
prog.write_reg(dst='r3',  src=12345678)
prog.write_reg(dst='r4',  src=12345678)
prog.write_reg(dst='r5',  src=12345678)
prog.write_reg(dst='r6',  src=12345678)
prog.write_reg(dst='r7',  src=12345678)
prog.wait(t=1)
prog.write_reg(dst='r8',  src=12345678)
prog.write_reg(dst='r9',  src=12345678)
prog.write_reg(dst='r10', src=12345678)
prog.write_reg(dst='r11', src=12345678)
prog.write_reg(dst='r12', src=12345678)
prog.write_reg(dst='r13', src=12345678)
prog.write_reg(dst='r14', src=12345678)
prog.write_reg(dst='r15', src=12345678)
# Write all the Data registers to 0
prog.wait(t=2)
prog.write_reg(dst='r0',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r1',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r2',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r3',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r4',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r5',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r6',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r7',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r8',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r9',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r10', src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r11', src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r12', src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r13', src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r14', src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r15', src='s_zero')

prog.label('time_test')
prog.wait(t=3)
# Test the Dispatcher execution time
for N in range(11,31):
    prog.write_reg(dst='r15', src=N)
    prog.trigger(ros=[0], pins=[0], t=prog.cycles2us(2**N+15))

# Increase usr_time to test upper time bits
# Need to force internal register because tProc can't write it directly
# use an internal register to sync the testbench
# Wait for time to be 2**30
prog.wait(t=prog.cycles2us(2**30))
# Increase reference to 2**30
prog.delay(t=prog.cycles2us(2**30))

for N in range(31,49):
    prog.write_reg(dst='r15', src=N)
    # Force reference from testbench to be 2**N
    # prog.delay(t=prog.cycles2us(2**N))
    # Wait 1us
    prog.wait(t=1)
    # Push trigger after another 1us
    prog.trigger(ros=[0], pins=[0], t=1)
prog.end()


print(prog)
prog.print_pmem2hex()


# Test issue 359

In [ ]:
from qick.asm_v2 import QickProgramV2, AsmInst

prog = QickProgramV2(soccfg)
prog.declare_readout(ch=0, length=100)
prog.label('arith_test')
prog.write_reg(dst='r0',  src=10)
prog.write_reg(dst='r1',  src=3)
# inst = AsmInst(inst={'CMD':"REG_WR", 'DST': 's2', 'SRC':'imm', 'LIT':'#49'}, addr_inc=1)
# prog.append_macro(inst)
# inst = AsmInst(inst={'CMD':"ARITH", 'C_OP': 'T', 'R1':'r0', 'R2':'r1'}, addr_inc=1)
# prog.append_macro(inst)
# inst = AsmInst(inst={'CMD': 'JUMP', 'IF': 'NF', 'ADDR': '&5'}, addr_inc=1)
# prog.append_macro(inst)
inst = AsmInst(inst={'CMD': 'PA', 'C_OP': '3', 'R1':'r0', 'R2':'r1'}, addr_inc=1)
prog.append_macro(inst)
prog.nop()
prog.nop()
prog.nop()
prog.nop()
prog.nop()
inst = AsmInst(inst={'CMD': 'PB', 'C_OP': '3', 'R1':'s6', 'R2':'r0', 'R3':'r1'}, addr_inc=1)
prog.append_macro(inst)
prog.nop()
prog.nop()
prog.nop()
prog.nop()
prog.nop()
prog.write_reg(dst='r2',  src=1234)
prog.write_reg(dst='r3',  src='s6')
prog.write_reg(dst='r4',  src='s7')

prog.end()

print(prog)

prog.print_pmem2hex()

# Qubit Emulator 
Frequency Sweep

In [ ]:
class FrequencySweepProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        
        self.declare_gen(ch=gen_ch, nqz=1)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])

        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['freq'], gen_ch=gen_ch)

        # self.add_gauss(ch=gen_ch, name="ramp", sigma=cfg['ramp_len']/10, length=cfg['ramp_len'], even_length=True)
        self.add_pulse(ch=gen_ch, name="mypulse", ro_ch=ro_ch, 
                       style="const", 
                    #    envelope="ramp", 
                       freq=cfg['freq'], 
                       length=cfg['flat_len'],
                       phase=cfg['phase'],
                       gain=cfg['gain'], 
                      )

        self.add_loop("freq_sweep", self.cfg["steps"])

    def _body(self, cfg):
        # send the config to the dynamic RO
        self.send_readoutconfig(ch=cfg['ro_ch'], name="myro", t=0)
        self.pulse(ch=cfg['gen_ch'], name="mypulse", t=0.1)
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'])

In [ ]:
GEN_CH = 0
RO_CH = 0
FREQ = 100
DELTA_FREQ = 5

# do a sweep with STEPS points and plot decimated
config = {'steps': 21,
          'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'freq': QickSweep1D("freq_sweep", FREQ-DELTA_FREQ, FREQ+DELTA_FREQ),
          'trig_time': 0.5,
          'ro_len': 2.0,
          'flat_len': 1.0,
        #   'ramp_len': 0.2,
          'phase': 0,
          'gain': 0.5
         }

# prog = FrequencySweepProgram(soccfg, reps=1, final_delay=0.5, cfg=config)
# iq_list = prog.acquire_decimated(soc, rounds=1)

# plt.figure()
# t = prog.get_time_axis(ro_index=0)
# for ii, iq in enumerate(iq_list[0]):
#     # plt.plot(t, iq[:,0], label="I value, step %d"%(ii))
#     # plt.plot(t, iq[:,1], label="Q value, step %d"%(ii))
#     plt.plot(t, np.abs(iq.dot([1,1j])), label="mag, step %d"%(ii))
# plt.legend()
# plt.ylabel("a.u.")
# plt.xlabel("us")


prog = FrequencySweepProgram(soccfg, reps=1, final_delay=0.5, cfg=config)
freqs = prog.get_pulse_param('myro', 'freq', as_array=True)
print(freqs)
iq_list = prog.acquire(soc, rounds=1, progress=True)

plt.figure()
plt.plot(freqs, np.abs(iq_list[0][0].dot([1,1j])))
# plt.plot(np.angle(iq_list[0][0].dot([1,1j]), deg=True))
# plt.plot(iq_list[0][0,:,0], iq_list[0][0,:,1])
plt.ylabel("amp")
plt.xlabel("freq")



In [ ]:
print(prog)
prog.print_pmem2hex()
prog.print_wmem2hex()
prog.print_sg_mem(sg_idx=0,gen_file=True)